In [2]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

pdf_path = r"D:\hugging_face\algorithm.pdf"
text_data = extract_text_from_pdf(pdf_path)


Previous trailer can not be read ("'NumberObject' object is not subscriptable",)
Object ID 7599,0 ref repaired
Object 7309 0 found
Object ID 7601,0 ref repaired
Object 1 0 found
Object 3 0 found
Object 6 0 found
Object 8 0 found
Object 11 0 found
Object 14 0 found
Object 17 0 found
Object 20 0 found
Object 23 0 found
Object 26 0 found
Object 29 0 found
Object 32 0 found
Object 35 0 found
Object 37 0 found
Object 40 0 found
Object 43 0 found
Object 46 0 found
Object 49 0 found
Object 52 0 found
Object 55 0 found
Object 58 0 found
Object 61 0 found
Object 63 0 found
Object 66 0 found
Object 69 0 found
Object 72 0 found
Object 75 0 found
Object 78 0 found
Object 81 0 found
Object 84 0 found
Object 87 0 found
Object 90 0 found
Object 93 0 found
Object 103 0 found
Object 106 0 found
Object 109 0 found
Object 112 0 found
Object 115 0 found
Object 118 0 found
Object 121 0 found
Object 139 0 found
Object 143 0 found
Object 146 0 found
Object 149 0 found
Object 152 0 found
Object 155 0 found
Ob

In [3]:
import json

data = [{"text": chunk} for chunk in text_data.split("\n\n")]
with open("dataset.jsonl", "w") as f:
    for item in data:
        f.write(json.dumps(item) + "\n")

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset

# 1. Define model path and tokenizer
model_name = r"D:\hugging_face\llama_3.2-1b_Model"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2. Define pad token (if it's missing)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Using EOS token as padding token

# 3. Prepare model for k-bit quantization (instead of int8)
model = prepare_model_for_kbit_training(model)

# 4. Configure LoRA (Low-Rank Adaptation)
lora_config = LoraConfig(
    r=8,  # Rank for LoRA matrices
    lora_alpha=32,  # Scaling factor for LoRA
    lora_dropout=0.1,  # Dropout for LoRA layers
    task_type="CAUSAL_LM"  # Type of task (Causal Language Modeling)
)

# 5. Apply LoRA model adapter
model = get_peft_model(model, lora_config)

# 6. Load dataset from JSON file
dataset = load_dataset("json", data_files="dataset.jsonl", split="train")

# 7. Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# 8. Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 9. Save the fine-tuned model
output_dir = "path_to_save_finetuned_model"  # Change this to the desired save location
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Model and tokenizer saved to path_to_save_finetuned_model


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 1. Load the fine-tuned model and tokenizer
model_path = "path_to_save_finetuned_model"  # Replace with your saved model's path
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# 2. Add a padding token if it's not set
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

# 3. Define a function to generate responses
def generate_response(prompt, max_length=200):
    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    # Generate output
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,  # Provide attention mask for reliable results
        max_length=max_length,
        num_beams=5,  # Beam search for better quality
        no_repeat_ngram_size=2,  # Avoid repetitive phrases
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# 4. Interactive Q&A loop
def ask_question():
    print("Chatbot is ready to ask questions. Type 'exit' to quit.")
    while True:
        # Ask user a question
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Goodbye!")
            break
        
        # Generate response
        prompt = f"Answer this question: {user_input}"
        response = generate_response(prompt)
        print(f"Bot: {response}")

# 5. Start the Q&A
ask_question()


Chatbot is ready to ask questions. Type 'exit' to quit.
Bot: Answer this question: who is ronaldo's father?
Who is Ronaldo’s father? Ronaldo is a Portuguese professional footballer who plays as a forward for Spanish club Real Madrid and the Portugal national team. He is widely regarded as one of the greatest footballers of all time. Ronaldo was born in Funchal, Madeira, Portugal, on February 5, 1985. His father, Jorge Mendes, was a businessman and his mother, Maria de Lourdes Pintasilgo, worked as an administrative assistant in a bank. As a child, Ronaldo showed an early interest in football and began playing for local teams at the age of six. In 1996, he joined the youth team of Benfica, where he made his professional debut in 2002. Over the next few years he played for several clubs in Portugal and Spain, including Sporting CP, Real Betis, and Sevilla FC. During this time he also represented Portugal at various youth levels,
Bot: Answer this question: what is sorting in computer scie